# Bufstream Iceberg quickstart

In [ ]:
from pyspark.sql import SparkSession

conf = (
    pyspark.SparkConf()
        .setAppName('Jupyter')
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .set("spark.sql.catalog.bufstream-quickstart", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.bufstream-quickstart.type", "rest")
        .set("spark.sql.catalog.bufstream-quickstart.uri", "http://iceberg-rest:8181")
)
spark = SparkSession.builder.config(conf=conf).getOrCreate()


In [6]:
%%sql
WITH categories AS (
    SELECT 
        line_item.product.category.name AS category,
        SUM(line_item.quantity * line_item.unit_price_cents) / 100.0 AS revenue,
        COUNT(DISTINCT val.cart_id) AS carts,
        SUM(line_item.quantity) AS units_sold
    FROM `bufstream`.`orders`
    LATERAL VIEW EXPLODE(val.line_items) AS line_item
    GROUP BY category
),
totals AS (
    SELECT 
        'TOTAL' AS category,
        SUM(revenue) AS revenue,
        -- We can't just SUM() carts: one cart may have items from multiple categories.
        (SELECT(COUNT(DISTINCT val.cart_id)) AS carts FROM `bufstream`.`orders`) AS carts,
        SUM(units_sold) AS units_sold
    FROM categories
)
SELECT
    category,
    FORMAT_NUMBER(revenue, 2) as revenue,
    FORMAT_NUMBER(carts, 0) as carts,
    FORMAT_NUMBER(units_sold, 0) as units_sold
FROM (
    SELECT * FROM categories
    UNION ALL
    SELECT * FROM totals
)    
ORDER BY 
    CASE WHEN category = 'TOTAL' THEN 1 ELSE 0 END,
    category DESC;


category,revenue,carts,units_sold
Sports & Outdoors,"2,911,510.93","15,710","74,407"
Personal Care,"1,148,661.57","12,446","50,143"
Kitchen & Dining,"2,339,124.98","14,262","62,302"
Home & Garden,"2,447,550.94","15,720","75,406"
Electronics & Accessories,"2,172,613.62","15,650","74,338"
Books & Stationery,"1,669,072.60","14,219","62,140"
TOTAL,"12,688,534.64","24,227","398,736"
